In [ ]:
!pip install pytorch-lightning


# **define the Network with PyTorch Lightning**

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

class LightningThreeLayerNet(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size):
        super(LightningThreeLayerNet, self).__init__()
        self.hidden1 = torch.nn.Linear(input_size, hidden_size)
        self.hidden2 = torch.nn.Linear(hidden_size, hidden_size)
        self.output = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        return self.output(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss)
        return loss


# **generate data and Create DataLoaders**

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl

# Set the random seed for reproducibility
pl.seed_everything(42)

# Parameters for the synthetic non-linear equation
a, b, c, d = 2, 1.5, -3, 0.5

# Generate synthetic data
x1 = np.random.uniform(-1, 1, 1000)
x2 = np.random.uniform(-1, 1, 1000)
x3 = np.random.uniform(-1, 1, 1000)
y = a * (x1 ** 2) + b * x2 + c * np.sin(x3) + d

# Combine x1, x2, x3 into a single array
X = np.vstack((x1, x2, x3)).T

# Split the data into training and testing sets
train_size = int(0.8 * len(X))  # 80% of data for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Convert the numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create datasets from tensors
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for the training and testing data
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


INFO:lightning_fabric.utilities.seed:Seed set to 42


# **Train with PyTorch Lightning's Trainer**

In [ ]:
# Set the dimensions for input, hidden, and output layers
input_size = 3  # Since we have three input features
hidden_size = 64  # Example hidden layer size
output_size = 1  # We're doing regression, so we need one output value

# Now instantiate the model
model = LightningThreeLayerNet(input_size, hidden_size, output_size)

# Then create the trainer and fit the model
# Remove or comment out the `progress_bar_refresh_rate` argument if it causes an error
trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, train_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type   | Params
-----------------------------------
0 | hidden1 | Linear | 256   
1 | hidden2 | Linear | 4.2 K 
2 | output  | Linear | 65    
-----------------------------------
4.5 K     Trainable params
0         Non-trainable params
4.5 K     Total params
0.018     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
# Run the test
trainer.test(model, dataloaders=test_loader)



Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.0007926042308099568   │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0007926042308099568}]